In [1]:
# pip install --upgrade python-telegram-bot

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 644.8/644.8 kB 16.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 3.7 MB/s eta 0:00:00


In [6]:
import logging
import torch
from transformers import GPT2Tokenizer, GPT2LMHeadModel
from telegram import Update
from telegram.ext import ApplicationBuilder, CommandHandler, MessageHandler, filters, ContextTypes
from google.colab import drive
import os
import nest_asyncio
import asyncio

In [3]:
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
model_path = '/content/drive/MyDrive/final_model.pth'


In [7]:
nest_asyncio.apply()

In [8]:
TOKEN = 'passwrd'

logging.basicConfig(
    format='%(asctime)s - %(name)s - %(levelname)s - %(message)s',
    level=logging.INFO
)
logger = logging.getLogger(__name__)

device = 'cuda' if torch.cuda.is_available() else 'cpu'
tokenizer = GPT2Tokenizer.from_pretrained('gpt2-medium')
tokenizer.pad_token = tokenizer.eos_token
model = GPT2LMHeadModel.from_pretrained('gpt2-medium')
model.to(device)
model.load_state_dict(torch.load(model_path, map_location=device), strict=False)
model.eval()

def generate_response(question, tokenizer, model, max_length=128):
    inputs = tokenizer.encode(question, return_tensors='pt').to(device)
    with torch.no_grad():
        outputs = model.generate(inputs, max_length=max_length, num_return_sequences=1, pad_token_id=tokenizer.eos_token_id)
    response = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return response

async def start(update: Update, context: ContextTypes.DEFAULT_TYPE) -> None:
    user = update.effective_user
    await update.message.reply_text(f'안녕하세요 {user.first_name}!\n질문을 입력해 주세요.')

async def help_command(update: Update, context: ContextTypes.DEFAULT_TYPE) -> None:
    await update.message.reply_text('도움이 필요하시면 이 봇에게 질문을 하세요.')

async def handle_message(update: Update, context: ContextTypes.DEFAULT_TYPE) -> None:
    user_input = update.message.text
    response = generate_response(user_input, tokenizer, model)
    await update.message.reply_text(response)

In [ ]:
def main() -> None:
    app = ApplicationBuilder().token(TOKEN).build()

    app.add_handler(CommandHandler("start", start))
    app.add_handler(CommandHandler("help", help_command))
    app.add_handler(MessageHandler(filters.TEXT & ~filters.COMMAND, handle_message))

    try:
        app.run_polling()
    except RuntimeError as e:
        if "This event loop is already running" in str(e):
            print("이벤트 루프가 이미 실행 중입니다.")
        else:
            raise

if __name__ == '__main__':
    main()

The attention mask is not set and cannot be inferred from input because pad token is same as eos token.As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
